In [1]:
from functools import cache

In [2]:
blueprints = list() # Remember to count from 1 when computing the quality level
with open('day_19_input.txt') as file:
    while line:= file.readline().rstrip():
        line = line.split(':')[1] # Leave only meaningful sentences
        robot_costs = line.split('.')
        ore_cost = robot_costs[0].split()
        clay_cost = robot_costs[1].split()
        obsidian_cost = robot_costs[2].split()
        geode_cost = robot_costs[3].split()
        blueprints.append({
            ore_cost[1]: {ore_cost[5]: int(ore_cost[4])},
            clay_cost[1]: {clay_cost[5]: int(clay_cost[4])},
            obsidian_cost[1]: {obsidian_cost[5]: int(obsidian_cost[4]),
                               obsidian_cost[8]: int(obsidian_cost[7])},
            geode_cost[1]: {geode_cost[5]: int(geode_cost[4]),
                            geode_cost[8]: int(geode_cost[7])}
        })

#### Task 1 (slow brute force)

An easy way to speed up the process would be to check if at least one geode can be created within the 24 minutes with a blueprint and omitting any other computations if it cannot:

In [3]:
opt_geodes = list()
for blueprint in blueprints:
    robots = (1, 0, 0, 0)
    supplies = (0, 0, 0, 0)
    ore_ore = blueprint['ore']['ore']
    cl_ore = blueprint['clay']['ore']
    obs_ore, obs_cl = blueprint['obsidian']['ore'], blueprint['obsidian']['clay']
    geo_ore, geo_obs = blueprint['geode']['ore'], blueprint['geode']['obsidian']

    @cache
    def planning_step(robots, supplies, time_remaining):
        if time_remaining == 0:
            return supplies[3]

        solutions = list()

        # If a geode robot can be built, it should!
        if supplies[0] >= geo_ore and supplies[2] >= geo_obs:
            return planning_step((robots[0], robots[1], robots[2], robots[3] + 1),
                                 (supplies[0] + robots[0] - geo_ore,
                                  supplies[1] + robots[1],
                                  supplies[2] + robots[2] - geo_obs,
                                  supplies[3] + robots[3]),
                                 time_remaining - 1)
        
        # Then, if a obsidian robot can be built, it should
        elif supplies[0] >= obs_ore and supplies[1] >= obs_cl and robots[2] < geo_obs:
            solutions.append(planning_step((robots[0], robots[1], robots[2] + 1, robots[3]),
                                           (supplies[0] + robots[0] - obs_ore,
                                            supplies[1] + robots[1] - obs_cl,
                                            supplies[2] + robots[2],
                                            supplies[3] + robots[3]),
                                           time_remaining - 1))
            
        # Then, if a clay robot can be built, it should
        elif supplies[0] >= cl_ore and robots[1] < obs_cl:
            solutions.append(planning_step((robots[0], robots[1] + 1, robots[2], robots[3]),
                                           (supplies[0] + robots[0] - cl_ore,
                                            supplies[1] + robots[1],
                                            supplies[2] + robots[2],
                                            supplies[3] + robots[3]),
                                           time_remaining - 1))
            
        # One should also consider building an ore robot
        if supplies[0] >= ore_ore and robots[0] < max(cl_ore, obs_ore, geo_ore):
            solutions.append(planning_step((robots[0] + 1, robots[1], robots[2], robots[3]),
                                           (supplies[0] + robots[0] - ore_ore,
                                            supplies[1] + robots[1],
                                            supplies[2] + robots[2],
                                            supplies[3] + robots[3]),
                                           time_remaining - 1))
    
        # Finally, not doing anything must also be considered!
        solutions.append(planning_step((robots[0], robots[1], robots[2], robots[3]),
                                       (supplies[0] + robots[0],
                                        supplies[1] + robots[1],
                                        supplies[2] + robots[2],
                                        supplies[3] + robots[3]),
                                       time_remaining - 1))
        return max(solutions)

    opt_geodes.append(planning_step(robots, supplies, time_remaining=24))

In [4]:
# Task 1
sum([(i + 1) * opt_geodes[i] for i in range(len(opt_geodes))])

1081

#### Task 2 (integer programming)

In [5]:
from scipy.optimize import linprog
import numpy as np

In [6]:
three_opt_geodes = list()
for blueprint in blueprints[:3]:
    ore_ore = blueprint['ore']['ore']
    cl_ore = blueprint['clay']['ore']
    obs_ore, obs_cl = blueprint['obsidian']['ore'], blueprint['obsidian']['clay']
    geo_ore, geo_obs = blueprint['geode']['ore'], blueprint['geode']['obsidian']

    time_remaining = 32 + 1 # + 1 for the start!
    A = list()
    b = list()
    robot_types = 4
    for time in range(time_remaining):
        # Can only append 1 robot a time:
        state_cost = np.zeros((time_remaining, robot_types))
        state_cost[time] = 1
        A.append(state_cost.flatten())
        b.append(1)

        # There must have been enough ore to sustain the robot population:
        state_cost = np.zeros((time_remaining, robot_types))
        state_cost[:(time + 1), 0] = ore_ore
        state_cost[:(time + 1), 1] = cl_ore
        state_cost[:(time + 1), 2] = obs_ore
        state_cost[:(time + 1), 3] = geo_ore
        state_cost[:time, 0] -= np.arange(time - 1, -1, -1)
        A.append(state_cost.flatten())
        b.append(ore_ore) # to allow for the first ore robot

        # There must have been enough clay to sustain the robot population:
        state_cost = np.zeros((time_remaining, robot_types))
        state_cost[:(time + 1), 2] = obs_cl
        state_cost[:time, 1] -= np.arange(time - 1, -1, -1)
        A.append(state_cost.flatten())
        b.append(0)

        # There must have been enough obsidian to sustain the robot population:
        state_cost = np.zeros((time_remaining, robot_types))
        state_cost[:(time + 1), 3] = geo_obs
        state_cost[:time, 2] -= np.arange(time - 1, -1, -1)
        A.append(state_cost.flatten())
        b.append(0)

    A = np.array(A)
    b = np.array(b)
    bounds = tuple((0, 1) for i in range(len(b)))
    state_reward = np.zeros((time_remaining, robot_types))
    state_reward[:, 3] = - np.arange(time_remaining - 1, -1, -1) # minus as linprog minimizes
    c = state_reward.flatten()
    res = linprog(c=c, A_ub=A, b_ub=b, bounds=bounds, integrality=[1 for i in range(len(b))])
    three_opt_geodes.append(int(-res['fun']))

In [7]:
# Task
np.array(three_opt_geodes).prod()

2415